# Notes on using the SIR-X model using DiseaseOutbreak.jl

First install Julia from [Julia website](https://julialang.org/downloads).

We are assuming you have Jupyter already installed, and you need to add the Julia kernel.  Follow the instructions in the [IJulia website](https://github.com/JuliaLang/IJulia.jl).

Next install the DiseaseOutbreak package.  It is not officially registered, so you install it directly from the website.

In [ ]:
using Pkg
Pkg.add(PackageSpec(url="https://github.com/senresearch/DiseaseOutbreak.jl"))

You load the package by doing this (it might take a little time).

In [ ]:
using DiseaseOutbreak

Load the CSV package for reading in CSV files.

In [ ]:
using CSV

Load text file with case counts from Hubei.

In [ ]:
# Hubei file name
hubeiFile = joinpath(@__DIR__, "..", "data",
     "covid19_china","time_series_covid19_confirmed_hubei.csv")
# read file into data frame
hubei = CSV.read(hubeiFile)

In [ ]:
# population of Hubei
hubeiPop = 57.0e6

Julia is type sensitive; since the data are counts, we have to convert them to a floating point vector.  We take 22 points, which are the points considered in the paper.

In [ ]:
# convert to floating point -- right way
# hubeiC = convert(Vector{Float64},hubei.ConfirmedCases[1:22])
hubeiC = 1.0*hubei.ConfirmedCases[1:22]

Next we fit the SIRX model to the case number series.

- time series length: 22
- case number series: hubeiC
- population size: hubeiPop
- R0Free: 6.2
- TInfected: 8
- initial values of parameters [κ,κ0,IXRatio]; all should be positive

In [ ]:
hubeiFit = fitCaseModel(22,hubeiC,hubeiPop,6.2,8.0,[0.1,0.1,2.0]);

Let us get a summary of the estimates.

In [ ]:
summary(hubeiFit)

Plot the case series along with the fitted curve.  We extend the prediction to 40 days; we had data for 22 days.

In [ ]:
plotfit(hubeiFit,40)

Now we can analyze Shandong.

In [ ]:
# File name (full path)
shandongFile = joinpath(@__DIR__, "..", "data",
          "covid19_china","time_series_covid19_confirmed_shandong.csv")
# read case counts
shandong = CSV.read(shandongFile)
# population
shandongPop = 94.2e6

# Convert case counts
shandongC = 1.0*shandong.ConfirmedCases[1:30]
shandongFit = fitCaseModel(30,shandongC,shandongPop,
                           6.2,8.0,[0.5,0.045,15.0])

In [ ]:
summary(shandongFit)

In [ ]:
plotfit(shandongFit,40)

In [ ]:
plot!(estimatedStates(30,shandongPop,shandongC[1],9.66,
                getParams(0.309,0.042,6.2,8.0,SIRX()))[:X],
                yaxis=:log,label="paper")

fitted(shandongFit,40)